In [ ]:
import pandas as pd
df=pd.read_csv("car data.csv")
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [ ]:
# One-Hot Encoding for categorical variables
df = pd.get_dummies(df, columns=['Fuel_Type', 'Selling_type', 'Transmission'], drop_first=True)



In [ ]:
# Drop the 'Car_Name' column as it is not needed for the model
df = df.drop(columns=['Car_Name'])

# Display the cleaned data
print(df.head())


   Year  Selling_Price  Present_Price  Driven_kms  Owner  Fuel_Type_Diesel  \
0  2014           3.35           5.59       27000      0             False   
1  2013           4.75           9.54       43000      0              True   
2  2017           7.25           9.85        6900      0             False   
3  2011           2.85           4.15        5200      0             False   
4  2014           4.60           6.87       42450      0              True   

   Fuel_Type_Petrol  Selling_type_Individual  Transmission_Manual  
0              True                    False                 True  
1             False                    False                 True  
2              True                    False                 True  
3              True                    False                 True  
4             False                    False                 True  


In [ ]:
from sklearn.model_selection import train_test_split

# Define the features and target
X = df.drop(columns=['Selling_Price'])
y = df['Selling_Price']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.svm import SVR
# Initialize SVR model with RBF kernel
svr_model = SVR(kernel='rbf', C=100, gamma=0.01, epsilon=0.1)

# Train the model
svr_model.fit(X_train, y_train)



SVR(C=100, gamma=0.01)

In [ ]:
# Predict on the test set
y_pred_svr = svr_model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred_svr):.2f}")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_svr):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred_svr):.2f}")


Mean Absolute Error (MAE): 0.64
Mean Squared Error (MSE): 1.10
R² Score: 0.95


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid
param_grid = {
    'C': [1, 10, 100],
    'gamma': [0.01, 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=5, scoring='r2')

# Train using GridSearch
grid_search.fit(X_train, y_train)

# Best parameters found
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Train the best SVR model
best_svr = grid_search.best_estimator_

# Predict again with optimized SVR
y_pred_best_svr = best_svr.predict(X_test)

# Evaluate
print(f"Optimized SVR R² Score: {r2_score(y_test, y_pred_best_svr):.2f}")


Best Hyperparameters: {'C': 100, 'epsilon': 0.1, 'gamma': 0.01}
Optimized SVR R² Score: 0.95


OUT OF SAMPLE PREDICTION

In [ ]:

# Define a new car's details as a DataFrame
new_data = pd.DataFrame({
    'Year': [2018],
    'Present_Price': [8.50],
    'Driven_kms': [35000],
    'Fuel_Type': ['Petrol'],
    'Selling_type': ['Dealer'],
    'Transmission': ['Manual'],
    'Owner': [0]
})

# Apply the same preprocessing steps (one-hot encoding)
new_data = pd.get_dummies(new_data, columns=['Fuel_Type', 'Selling_type', 'Transmission'], drop_first=True)

# Ensure all columns match training data
for col in X.columns:
    if col not in new_data:
        new_data[col] = 0  # Add missing columns as 0

# Reorder columns to match the training set
new_data = new_data[X.columns]


In [ ]:
# Scale the new data using the trained scaler
new_data_scaled = scaler.transform(new_data)


In [ ]:
# Predict the price using the trained SVR model
predicted_price = svr_model.predict(new_data_scaled)

print(f"Predicted Selling Price: {predicted_price[0]:.2f} Lakhs")


Predicted Selling Price: 8.34 Lakhs
